In [4]:

metrics = [
	{
	'group': 'cpu',
	'name': 'cpu_seconds_user',
	'ylabel': 'Percentage',
    'ylim': [0, 100],
    'xlabel': 'Epoch',
    'title': 'CPU consumption',
    'queries': [
			{
				'query': f"""rate(node_cpu_seconds_total[{step}]) * 100""",
				'filters': {
					'mode': 'user'
					}
			}
		],
	},
	{
	'group': 'memory',
	'name': 'used_mem',
    'ylabel': 'Percentage',
    'ylim': [0, 100],
    'xlabel': 'Epoch',
    'title': 'Used Memory',
    'queries': [
			{
				'query': f"""(node_memory_MemTotal_bytes - node_memory_MemAvailable_bytes) / node_memory_MemTotal_bytes * 100""",
				'filters': {}
			}
		],
	},
    {
	'group': 'network',
	'name': 'network_receive',
    'ylabel': f"""MB/{step}""",
    'ylim': [0, 10],
    'xlabel': 'Epoch',
    'title': 'Network Receive Rate',
    'queries': [
			{
				'query': f"""rate(node_network_receive_bytes_total[{step}]) / 1024 / 1024""",
				'filters': {
					'device': 'eno1'
					}
			}
		],
	},
    {
	'group': 'network',
	'name': 'network_transmit',
    'ylabel': f"""MB/{step}""",
    'ylim': [0, 2],
    'xlabel': 'Epoch',
    'title': 'Network Send Rate',
    'queries': [
			{
				'query': f"""rate(node_network_transmit_bytes_total[{step}]) / 1024 / 1024""",
				'filters': {
					'device': 'eno1'
					}
			}
		],
	},
    {
	'group': 'peers',
	'name': 'libp2p_peers',
    'ylabel': f"""Count""",
    'ylim': [0, 200],
    'xlabel': 'Epoch',
    'title': 'Lib P2P Peers',
    'queries': [
			{
				'query': f"""libp2p_peers""",
				'filters': {}
			},
            {
				'query': f"""p2p_peer_count""",
				'filters': {
                    'state': 'Connected'
				}
			}
		],
	},
    {
	'group': 'disk',
	'name': 'disk_write_bytes',
    'ylabel': f"""MB/m""",
    'ylim': [0, 10],
    'xlabel': 'Epoch',
    'title': 'Disk Write Bytes Ratio',
    'queries': [
			{
				'query': f"""rate(node_disk_written_bytes_total[{step}]) / 1024 / 1024""",
				'filters': {
                    'device': "nvme0n1"
				}
			}
		],
	},
    {
	'group': 'disk',
	'name': 'disk_read',
    'ylabel': f"""MB/m""",
    'ylim': [0, 10],
    'xlabel': 'Epoch',
    'title': 'Disk Read Bytes Ratio',
    'queries': [
			{
				'query': f"""rate(node_disk_read_bytes_total[{step}]) / 1024 / 1024""",
				'filters': {
                    'device': "nvme0n1"
				}
			}
		],
	},
    {
	'group': 'disk',
	'name': 'disk_read_ops',
    'ylabel': f"""OPS/m""",
    'ylim': [0, 10],
    'xlabel': 'Epoch',
    'title': 'Disk Read Ops Ratio',
    'queries': [
			{
				'query': f"""rate(node_disk_reads_completed_total[{step}]) / 1024 / 1024""",
				'filters': {
                    'device': "nvme0n1"
				}
			}
		],
	},
    {
	'group': 'disk',
	'name': 'disk_write_ops',
    'ylabel': f"""OPS/m""",
    'ylim': [0, 10],
    'xlabel': 'Epoch',
    'title': 'Disk Write Ops Ratio',
    'queries': [
			{
				'query': f"""rate(node_disk_writes_completed_total[{step}]) / 1024 / 1024""",
				'filters': {
                    'device': "nvme0n1"
				}
			}
		],
	},
]


# Metrics

# METRIC_MEMORY = 'node_memory_MemAvailable_bytes'
# METRIC_MEMORYTOTAL = 'node_memory_MemTotal_bytes'
# METRIC_CPU = 'node_cpu_seconds_total'
# METRIC_NETRECV = 'node_network_receive_bytes_total'
# METRIC_NETSENT = 'node_network_transmit_bytes_total'
# METRIC_NETRECV_PACK = 'node_network_receive_packets_total'
# METRIC_NETSENT_PACK = 'node_network_transmit_packets_total'
# METRIC_SLOT = 'beacon_head_slot'
# METRIC_PEERS = 'libp2p_peers'
# METRIC_DISKTOTAL = "node_filesystem_size_bytes"
# METRIC_DISKAVAIL = "node_filesystem_avail_bytes"
# METRIC_DISKWRITE_BYTES = "node_disk_written_bytes_total"
# METRIC_DISKREAD_BYTES = "node_disk_read_bytes_total"
# METRIC_DISKWRITE_OPS = "node_disk_writes_completed_total"
# METRIC_DISKREAD_OPS = "node_disk_reads_completed_total"
# METRIC_EARNS = "validators_epoch_earned_amount_metrics"



In [ ]:
# CPU

from base import *
import plotly.graph_objects as go

metric = 'cpu'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='CPU Usage',
	xaxis_title="Date",
	yaxis_title="Percentage",
)
fig.show()
fig.write_html(fig_folder + 'cpu.html')

        

In [ ]:
# CPU per slot second

from base import *
import plotly.graph_objects as go

metric = "cpu_per_second_default"
df = import_csv(metric)
df['slot_second'] = (df['timestamp'] - genesis_unix_time) % 60

fig = go.Figure()
for client, data in df.groupby('client'):
		data_agg = data.groupby(by='slot_second').agg({
			'value': 'mean'
		}).reset_index()
		fig.add_trace(
			go.Scatter(
				x=data_agg["slot_second"], 
				y=data_agg["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='CPU Usage Default',
	xaxis_title="Slot Second",
	yaxis_title="Percentage",
	xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 12
    )
)
fig.show()
fig.write_html(fig_folder + metric + '.html')


In [ ]:
# CPU per slot second all topics

from base import *
import plotly.graph_objects as go

metric = "cpu_per_second_all_topics"
df = import_csv(metric)
df['slot_second'] = (df['timestamp'] - genesis_unix_time) % 60

fig = go.Figure()
for client, data in df.groupby('client'):
		data_agg = data.groupby(by='slot_second').agg({
			'value': 'mean'
		}).reset_index()
		fig.add_trace(
			go.Scatter(
				x=data_agg["slot_second"], 
				y=data_agg["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='CPU Usage All Topics',
	xaxis_title="Slot Second",
	yaxis_title="Percentage",
	xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 12
    )
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# MEM

from base import *
import plotly.graph_objects as go


metric = "mem"

df = import_csv(metric)
df = df.sort_values(by='timestamp')

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title=f"""Memory Usage""",
	xaxis_title="Date",
	yaxis_title="GB",
)
fig.show()
fig.write_html(fig_folder + 'mem.html')

In [ ]:
# MEM without rss

from base import *
import plotly.graph_objects as go


metric = "mem_without_rss"

df = import_csv(metric)
df = df.sort_values(by='timestamp')

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title=f"""Memory Usage without RSS""",
	xaxis_title="Date",
	yaxis_title="GB",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# Network

from base import *
import plotly.graph_objects as go

metric = 'net_receive'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='Network In Rate',
	xaxis_title="Date",
	yaxis_title="MB/s",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# Network Receive Packets

from base import *
import plotly.graph_objects as go

metric = 'net_receive_packets'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='Network In Packets Rate per Second',
	xaxis_title="Date",
	yaxis_title="Number",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# Net Receive per slot second default

from base import *
import plotly.graph_objects as go

metric = "net_receive_per_second_default"
df = import_csv(metric)

df['slot_second'] = (df['timestamp'] - genesis_unix_time) % 60

fig = go.Figure()
for client, data in df.groupby('client'):
		data_agg = data.groupby(by='slot_second').agg({
			'value': 'mean'
		}).reset_index()
		fig.add_trace(
			go.Scatter(
				x=data_agg["slot_second"], 
				y=data_agg["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='Net Receive Default',
	xaxis_title="Slot Second",
	yaxis_title="MB/s",
	xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 12
    )
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# Net Receive per slot second all-topics

from base import *
import plotly.graph_objects as go

metric = "net_receive_per_second_all_topics"
df = import_csv(metric)

df['slot_second'] = (df['timestamp'] - genesis_unix_time) % 60

fig = go.Figure()
for client, data in df.groupby('client'):
		data_agg = data.groupby(by='slot_second').agg({
			'value': 'mean'
		}).reset_index()
		fig.add_trace(
			go.Scatter(
				x=data_agg["slot_second"], 
				y=data_agg["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='Net Receive All Topics',
	xaxis_title="Slot Second",
	yaxis_title="MB/s",
	xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 12
    )
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# Network Transmit

from base import *
import plotly.graph_objects as go

metric = 'net_transmit'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='Network Out Rate',
	xaxis_title="Date",
	yaxis_title="MB/s",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# Network Transmit Packets

from base import *
import plotly.graph_objects as go

metric = 'net_transmit_packets'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='Network Out Packets Rate per Second',
	xaxis_title="Date",
	yaxis_title="Number",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# Net transmit per slot second

from base import *
import plotly.graph_objects as go

df = import_csv('net_transmit_per_second_default')
df['slot_second'] = (df['timestamp'] - genesis_unix_time) % 12

fig = go.Figure()
for client, data in df.groupby('client'):
		data_agg = data.groupby(by='slot_second').agg({
			'value': 'mean'
		}).reset_index()
		fig.add_trace(
			go.Scatter(
				x=data_agg["slot_second"], 
				y=data_agg["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='Net Transmit',
	xaxis_title="Slot Second",
	yaxis_title="MB/s",
)
fig.show()

In [ ]:
# Peers

from base import *
import plotly.graph_objects as go

metric = 'peers'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='Peer Number',
	xaxis_title="Date",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# FS Read Bytes

from base import *
import plotly.graph_objects as go

metric = 'fs_read_bytes'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)
fig.update_yaxes(type='log')

fig.update_layout(
	title='FS Read Bytes',
	xaxis_title="Date",
	yaxis_title="KB/s",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# FS Read Ops

from base import *
import plotly.graph_objects as go

metric = 'fs_read_ops'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)
fig.update_yaxes(type='log')

fig.update_layout(
	title='FS Read Ops/s',
	xaxis_title="Date",
	yaxis_title="Number",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# FS Write Bytes

from base import *
import plotly.graph_objects as go

metric = 'fs_write_bytes'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)
fig.update_yaxes(type='log')

fig.update_layout(
	title='FS Write Bytes',
	xaxis_title="Date",
	yaxis_title="KB/s",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# FS Write Ops

from base import *
import plotly.graph_objects as go

metric = 'fs_write_ops'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)
fig.update_yaxes(type='log')

fig.update_layout(
	title='FS Write Ops/s',
	xaxis_title="Date",
	yaxis_title="Number",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# Disk Usage

from base import *
import plotly.graph_objects as go

metric = 'disk_usage'
df = import_csv(metric)

fig = go.Figure()
for client, data in df.groupby('client'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["value"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client])))

fig = decorate_fig(fig)

fig.update_layout(
	title='Disk Usage',
	xaxis_title="Date",
	yaxis_title="GB",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')

In [ ]:
# Block Proposals

from base import *
import plotly.graph_objects as go

metric = 'block_proposals'
df = import_blocks_csv(metric)
df['epoch'] = df['f_slot'] // 32
df['epoch_time'] = (df['epoch'] * 384) + genesis_unix_time
df['date'] = pd.to_datetime(df['epoch_time'], unit='s')

df_grouped = df.groupby(['date', 'epoch', 'f_client_name']).agg(
	proposed_blocks=pd.NamedAgg(column="f_score", aggfunc=lambda x: (x>0).sum()),
	missed_blocks=pd.NamedAgg(column="f_score", aggfunc=lambda x: (x<0).sum()),
).reset_index()

fig = go.Figure()
for client, data in df_grouped.groupby('f_client_name'):
		fig.add_trace(
			go.Scatter(
				x=data["date"], 
				y=data["proposed_blocks"], 
				name=client, 
				mode='lines', 
				line=go.scatter.Line(color=colors[client.lower()])))

fig = decorate_fig(fig)

fig.update_layout(
	title='Proposed Blocks Per Epoch',
	xaxis_title="Date",
)
fig.show()
fig.write_html(fig_folder + metric + '.html')